In [0]:
CREATE OR REPLACE FUNCTION shm.3w.predict_state(
    well_number_param INT DEFAULT -1 COMMENT "Well number between 1 and 50, must be integer. Select -1 to get all wells", 
    n_obs_param INT DEFAULT 10 COMMENT "Number of observations to predict"
)
RETURNS TABLE (
    well_number INTEGER,
    timestamp TIMESTAMP,
    prediction INTEGER 
)
COMMENT "Generates hydrate predictions for the latest sensor readings"
RETURN
SELECT 
    well_number,
    timestamp,
    AI_QUERY(
        'shm_3w_lightgbm',
        request => NAMED_STRUCT(
        'P-PDG', `latest_n_obs`.`P-PDG`,
        'P-TPT', `latest_n_obs`.`P-TPT`,
        'T-TPT', `latest_n_obs`.`T-TPT`,
        'P-MON-CKP', `latest_n_obs`.`P-MON-CKP`,
        'T-JUS-CKP', `latest_n_obs`.`T-JUS-CKP`,
        'QGL', `latest_n_obs`.`QGL`,
        'well_number', `latest_n_obs`.`well_number`
        ),
        returnType => 'DOUBLE'
    ) AS prediction
FROM shm.3w.latest_n_obs(well_number_param, n_obs_param)